In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


**Memanggil Library**

In [84]:
import numpy as np 
import random
from array import *

**Membuat daftar kota**

In [85]:
daftarKota = [[5,5], [10,25], [15, 15], [20,20],[25,10],[35,25],[30,5],[35,20],[40,10],[50,15]]
for i in daftarKota:
    for j in i:
        print(j,end = " ")
    print()

jumlahKota =  len(daftarKota)   
print(jumlahKota)

5 5 
10 25 
15 15 
20 20 
25 10 
35 25 
30 5 
35 20 
40 10 
50 15 
10


**Inisialisasi Populasi secara acak**

**Array P diinisialisasi sebagai array integer**

**Solusi awal dibuat dengan random permutation**


In [86]:
def initPopulasi(ukuranPopulasi,jumlahKota):
  P = np.empty((ukuranPopulasi,jumlahKota))
  P = P.astype('int')
  for i in range(ukuranPopulasi):
    I = np.random.permutation(jumlahKota)
    for j in range(jumlahKota):
      P[i][j]=I[j]
  return P


ukuranPopulasi = 4
P = initPopulasi (ukuranPopulasi,jumlahKota)
print(P)

[[8 0 6 4 3 1 7 5 2 9]
 [9 4 8 6 1 2 5 7 0 3]
 [7 1 5 4 9 0 2 6 3 8]
 [8 5 1 3 2 9 0 4 7 6]]


**Membuat fungsi objektif**

**Jarak yang diukur adalah jarak euclidian**


In [121]:
def fungsiObjectif(populasi,daftarKota):
  sz = populasi.shape
  ukuranPopulasi = sz[0]
  jumlahKota = sz[1]

  matrikJarak = np.empty((ukuranPopulasi))
  matrikJarak  = matrikJarak.astype('float')
  n = len(daftarKota)

  for i in range(ukuranPopulasi):
    jarak = 4
    for j in range(n-1):
      jrkX = daftarKota[populasi[i][j]][0]-daftarKota[populasi[i][j+1]][0]
      jrkY = daftarKota[populasi[i][j]][1]-daftarKota[populasi[i][j+1]][1]
      d = np.sqrt(jrkX**2 + jrkY**2 )   
      jarak = jarak  + d
      
    dn =  np.sqrt((daftarKota[populasi[i][n-1]][0]-daftarKota[populasi[i][0]][0])**2+ (daftarKota[populasi[i][n-1]][1]-daftarKota[populasi[i][0]][1])**2 )
    jarak = jarak  + dn
    matrikJarak[i] = jarak

  return matrikJarak

fitP = fungsiObjectif(P,daftarKota)
print(fitP)
#print(fitP.dtype)

[167.6075464  166.59199134 176.78254557 166.59199134]


**Seleksi menggunakan metode tournamen**

**Ukuran turnamen k = 2**

**Jumlah induk yang dipilih adalah 2**


In [122]:
def seleksi (P,fsObj):
  jmlInduk = 2
  sz = P.shape
  ukuranPopulasi = sz[0]
  jumlahKota = sz[1]

  Induk = np.empty((jmlInduk,jumlahKota))
  Induk = Induk.astype('int')
  
  for i in range (2):
    # memilih dua individu secara acak 
    r1 = random.randint(0,ukuranPopulasi-1)
    r2 = random.randint(0,ukuranPopulasi-1)
    
    if  fsObj[r1] < fsObj [r2]:
      for j in range(jumlahKota):
        Induk [i][j] = P[r1][j]
    else:
      for j in range(jumlahKota):
        Induk [i][j] = P[r2][j]
  return Induk

Induk = seleksi(P,fitP)
print(Induk)

[[4 1 3 0 4 7 6 8 9 5]
 [4 1 3 0 4 7 6 8 9 5]]


**Rekombinasi menggunaka metode Partially Mixed Crossover (PMX) Jumlah anak yang dihasilkan adalah 2**

In [123]:
#crossover 

def PMX(Induk):
  sz = Induk.shape
  brs = sz[0]
  kol = sz[1]
  #print(sz, brs, kol)

  Anak = np.empty((2,kol))
  Anak = Anak.astype('int')

  r1 = random.randint(1,kol-2)

  for i0 in range(r1): 

    Anak [0][i0] = Induk[0][i0]
    Anak [1][i0] = Induk[1][i0]

  batasCek1 = r1
  batasCek2 = r1

  for i in range(kol):
    m = (i+r1)%kol

    n1 = Induk[0][i] 
    n2 = Induk[1][i]

    #mengecek apakah kota tertentu sudah ada di anak
    idxAda1 = 0
    idxAda2 = 0
    for j1 in range(batasCek1): 
      
      if batasCek1>(kol-1):
        idxAda1 = 1

      if Anak [0][j1] ==n2: 
        idxAda1 = 1
            
    for j2 in range(batasCek2): 
      
      if batasCek2>(kol-1):
        idxAda2 = 1

      if Anak [1][i0]  == n1: 
        idxAda2 = 1

    if idxAda1 == 0:
      Anak[0][batasCek1] = n2
      batasCek1 = batasCek1 +1
   
    if idxAda2 == 0:
      Anak[1][batasCek2] = n1
      batasCek2 = batasCek2 +1
   
  return Anak

AnakCX = PMX(Induk)
print("Anak :",AnakCX)

Anak : [[4 1 3 0 4 7 6 8 9 5]
 [4 1 3 0 4 7 4 1 3 0]]


**Mutasi dengan Swap**

**index kota yang di swap pada kedua anak sama** 


In [124]:
def MutasiSwap(Induk):
  sz = Induk.shape
  brs = sz[0]
  kol = sz[1]

  Anak = np.empty((2,kol))
  Anak = Anak.astype('int')

  Anak = Induk

  r1 = random.randint(0,kol-1)
  r2 = random.randint(0,kol-1)

  while r1 == r2:
    r2 = random.randint(0,kol-1)

  I1rs1 = Induk [0][r1]
  I1rs2 = Induk [0][r2]
  I2rs1 = Induk [1][r1]
  I2rs2 = Induk [1][r2] 

  Anak [0][r1] = I1rs2
  Anak [0][r2] = I1rs1
  Anak [1][r1] = I2rs2  
  Anak [1][r2] = I2rs1

  return Anak

print("AnakCX :",AnakCX)
AnakM = MutasiSwap(AnakCX)
print("AnakM :",AnakM)

AnakCX : [[4 1 3 0 4 7 6 8 9 5]
 [4 1 3 0 4 7 4 1 3 0]]
AnakM : [[4 7 3 0 4 1 6 8 9 5]
 [4 7 3 0 4 1 4 1 3 0]]


**Elitism (steady state replacement)**

In [125]:
def Elitism(P,Anak,fitP,fitAnak):
  szP = P.shape
  brsP = szP[0]
  kolP = szP[1]
  szA = Anak.shape
  brsA = szA[0]
  kolA = szA[1]

  #print (fitP)
  for i in range(brsA):
    iJelek = fitP.max()
    idxJelek = fitP.argmax()

    if fitAnak[i]<iJelek:
      fitP[idxJelek] = fitAnak[i]
      for j in range (kolP):
        P[idxJelek][j] = Anak[i][j]

  return [P,fitP]

fitAnak= fungsiObjectif(AnakM,daftarKota)
print(P)
print(AnakM)
print(fitP)
print(fitAnak)
P = Elitism (P,AnakM,fitP,fitAnak)
print(P)
print(fitP)

[[4 1 3 0 4 1 3 7 6 8]
 [4 1 3 0 4 7 6 8 9 5]
 [4 1 3 0 4 7 6 8 4 1]
 [4 1 3 0 4 7 6 8 9 5]]
[[4 7 3 0 4 1 6 8 9 5]
 [4 7 3 0 4 1 4 1 3 0]]
[167.6075464  166.59199134 176.78254557 166.59199134]
[182.8845344  191.61954895]
[array([[4, 1, 3, 0, 4, 1, 3, 7, 6, 8],
       [4, 1, 3, 0, 4, 7, 6, 8, 9, 5],
       [4, 1, 3, 0, 4, 7, 6, 8, 4, 1],
       [4, 1, 3, 0, 4, 7, 6, 8, 9, 5]]), array([167.6075464 , 166.59199134, 176.78254557, 166.59199134])]
[167.6075464  166.59199134 176.78254557 166.59199134]


**Prosedur GA**

In [120]:
# Inisialisasi paramater
PCX = 0.95
Pm = 0.01
ukuranPopulasi = 4
max_generasi = 5

# Inisialisasi populasi 

P = initPopulasi (ukuranPopulasi,jumlahKota)
print("P0 :",P)

#melakukan evaluasi P
fitP = fungsiObjectif(P,daftarKota)
print("fitness populasi awal:",fitP)

print("fitness terbaik awal:",fitP.min())
print("rute awal :",P[fitP.argmin()])


for i in range(max_generasi):
  print("generasi :",i)
  Induk = seleksi(P,fitP)
  #print("Induk :",Induk)

  Anak = Induk

  rxo = random.random()

  if rxo < PCX:
    Anak = PMX(Induk)
    #print("AnakXO :",Anak)

    rm = random.random()
    if rm < Pm:
      Anak = MutasiSwap(Anak)
      #print("AnakR :",Anak)

  fitAnak = fungsiObjectif(Anak,daftarKota)
  #print("fitAnak :",fitAnak)

  print("fitness:",fitAnak.min())

  [P, fitP]= Elitism (P,Anak,fitP,fitAnak)   
  
print("fitness populasi akhir:",fitP)
print("fitness akhir:",fitP.min())
print("rute akhir :",P[fitP.argmin()])

P0 : [[8 4 5 0 7 3 6 9 2 1]
 [4 2 7 6 0 1 9 5 8 3]
 [3 1 9 4 7 2 6 0 5 8]
 [4 1 3 0 7 6 8 9 5 2]]
fitness populasi awal: [240.7340845  204.83400504 232.91949467 179.88861054]
fitness terbaik awal: 179.88861053684522
rute awal : [4 1 3 0 7 6 8 9 5 2]
generasi : 0
fitness: 179.88861053684522
generasi : 1
fitness: 179.88861053684522
generasi : 2
fitness: 166.6075463982163
generasi : 3
fitness: 165.59199134099075
generasi : 4
fitness: 165.59199134099075
fitness populasi akhir: [166.6075464  165.59199134 175.78254557 165.59199134]
fitness akhir: 165.59199134099075
rute akhir : [4 1 3 0 4 7 6 8 9 5]
